# Climate Change Modeling

In [ ]:
# 02 — Modeling

# --- Hybrid dataset loader ---
import os, io
import pandas as pd

DATA_PATH = "../data/clean_climate.csv"

if os.path.exists(DATA_PATH):
    print(f"Loading dataset from {DATA_PATH}")
    df = pd.read_csv(DATA_PATH)
else:
    try:
        from google.colab import files
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        df = pd.read_csv(io.BytesIO(uploaded[file_name]))
        print(f"Loaded uploaded file: {file_name}")
    except Exception as e:
        from tkinter import Tk, filedialog
        Tk().withdraw()
        file_path = filedialog.askopenfilename(filetypes=[("CSV files","*.csv")])
        df = pd.read_csv(file_path)
        print(f"Loaded local file: {file_path}")

# --- Modeling pipeline ---
import numpy as np
from src.data_preprocessing import basic_cleaning, train_val_test_split, scale_numeric
from src.modeling import train_random_forest, evaluate, save_artifacts
from src.feature_engineering import add_time_lags, add_rolling_features

df = basic_cleaning(df)

# Choose target
target = next((c for c in df.columns if 'anomaly' in c.lower() or 'target' in c.lower()),
              df.select_dtypes('number').columns.tolist()[0])
print('Target:', target)

# Optional time features
df = add_time_lags(df, target)
df = add_rolling_features(df, target)

# Keep numeric only and drop NAs for baseline
df_num = df.select_dtypes(include=[np.number]).dropna()
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(df_num, target)
Xtr, Xv, Xte, scaler = scale_numeric(X_train, X_val, X_test)

# Train RF
result = train_random_forest(Xtr, y_train, Xv, y_val)
print("Validation metrics:", result['val_metrics'])

In [ ]:
# Evaluate on test set
test_metrics = evaluate(result['model'], Xte, y_test)
print("Test metrics:", test_metrics)

In [ ]:
# Save artifacts
save_artifacts(result['model'], scaler, out_dir='../artifacts', prefix='rf')
print('Artifacts saved: ../artifacts/rf_model.pkl, ../artifacts/rf_scaler.pkl')